In [2]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.9,<4.0; 0.0.10rc0 Requires-Python >=3.9,<4.0; 0.0.11 Requires-Python >=3.9,<4.0; 0.0.1rc0 Requires-Python >=3.9,<4.0; 0.0.2 Requires-Python >=3.9,<4.0; 0.0.3 Requires-Python >=3.9,<4.0; 0.0.4 Requires-Python >=3.9,<4.0; 0.0.5 Requires-Python >=3.9,<4.0; 0.0.6 Requires-Python >=3.9,<4.0; 0.0.7 Requires-Python >=3.9,<4.0; 0.0.8 Requires-Python >=3.9,<4.0; 0.0.9 Requires-Python >=3.9,<4.0; 1.0.1 Requires-Python <4.0,>=3.9; 1.0.2 Requires-Python <4.0,>=3.9
ERROR: Could not find a version that satisfies the requirement langchain-google-genai (from versions: none)
ERROR: No matching distribution found for langchain-google-genai


In [4]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader   #Generic loaded to load any code base through langchain
from langchain.document_loaders.parsers import  LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface_hub import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import  ChatGooglePalm
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [5]:
## Clone Github Repos
%pwd

'c:\\Users\\nagan\\OneDrive\\Desktop\\Ineuron_Gen_AI_Course\\Projects\\In_Class_Projects\\Source-Code-Analyzer-Using-GenAI\\research'

In [6]:
!mkdir test_repo

In [7]:
Repo.clone_from('https://github.com/entbappy/End-to-end-ML-Project-Implementation.git',to_path='test_repo/')

<git.repo.base.Repo 'c:\\Users\\nagan\\OneDrive\\Desktop\\Ineuron_Gen_AI_Course\\Projects\\In_Class_Projects\\Source-Code-Analyzer-Using-GenAI\\research\\test_repo\\.git'>